Questions
1. How does total revenue change over time?
2. How does revenue differ by country?
3. What is the distribution of order amounts (winsorized)?
4. Is the refund rate different across countries?

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path

data1 = Path("../data/processed/analytics_table.parquet")
figuers1 = Path("reports/figures")
figuers1.mkdir(parents=True, exist_ok=True)




In [7]:
df = pd.read_parquet("../data/processed/analytics_table.parquet")
df.head(7)

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,A0001,1,12.5,1.0,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.500
1,A0002,2,8.0,2.0,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.135
2,A0003,3,NaN,1.0,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,NaN
3,A0004,1,25.0,NaN,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.000
4,A0005,4,100.0,1.0,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,97.750


In [8]:
df= pd.read_parquet(data1)
print("rows number : ",len(df))
print("cols number :", len(df.columns))
print(df.dtypes.head(17))

print(df.isna().sum().sort_values(ascending=False).head(10))

rows number :  5
cols number : 17
order_id               string[python]
user_id                         Int64
amount                        float64
quantity                      float64
created_at        datetime64[ns, UTC]
status                 string[python]
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
signup_date                    object
amount_winsor                 float64
dtype: object
amount           1
amount_winsor    1
quantity         1
created_at       1
hour             1
dow              1
month            1
year             1
date             1
order_id         0
dtype: int64


In [9]:
revenue = (
    df.groupby("month", dropna=False)
      .agg(
          revenue=("amount", "sum"),
          orders=("order_id", "count")
          
      )
      .reset_index()  
)

revenue


,month,revenue,orders
0,2025-12,45.5,4
1,<NA>,100.0,1


Revenue changes across months.


In [10]:
revenue2 = (
    df.groupby("country", dropna=False)
      .agg(
          revenue=("amount", "sum"),
          orders=("order_id", "count")
      )
      .reset_index()
      
)

revenue2

,country,revenue,orders
0,AE,0.0,1
1,SA,145.5,4


Some countries generate higher revenue than others.

In [11]:
from pathlib import Path
import pandas as pd

DATA = Path("../data/processed/analytics_table.parquet")

df = pd.read_parquet(DATA)
df.head()

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,A0001,1,12.5,1.0,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.500
1,A0002,2,8.0,2.0,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.135
2,A0003,3,NaN,1.0,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,NaN
3,A0004,1,25.0,NaN,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.000
4,A0005,4,100.0,1.0,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,97.750


In [12]:
import nbformat
import plotly.express as px


fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=30,
    title="Distribution of Order Amounts (Winsorized)"
)

fig.update_layout(
    title={"x": 0.02},
    xaxis_title="Order Amount (winsorized)",
    yaxis_title="Number of Orders"
)

fig

The dataset contains clean and structured transactional data with only a small number of missing values


In [13]:

df["is_refund"] = df["status_clean"].eq("refund")

rate = (
    df.groupby("country", dropna=False)
      .agg(
          refunds=("is_refund", "sum"),
          total=("is_refund", "count")
      )
      .assign(refund_rate=lambda x: x["refunds"] / x["total"])
      .reset_index()
)

rate


,country,refunds,total,refund_rate
0,AE,1,1,1.0
1,SA,0,4,0.0


The refund rate shows some variation across countries.  

In [14]:
a = df.loc[df["country"] == "SA", "is_refund"]
b = df.loc[df["country"] == "AE", "is_refund"]


In [15]:
import numpy as np

def bootstrap(a, b, n_boot=2000, seed=1):
    rng = np.random.default_rng(seed)

    a = a.dropna().to_numpy()
    b = b.dropna().to_numpy()

    differnt = []

    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        differnt.append(sa.mean() - sb.mean())

    return {
        "diff mean is=  ": float(a.mean() - b.mean()),
        "ci_low": float(np.percentile(differnt, 2.5)),
        "ci_high": float(np.percentile(differnt, 97.5)),
    }

result = bootstrap(a, b)
result

{'diff mean is=  ': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}

## Findings + Caveats


  Revenue varies across different time periods, with some months showing higher activity than others.  
  This suggests possible seasonality or changes in user behavior over time.


  Revenue differs across countries, with some countries contributing more to total revenue.  
  These differences appear to be driven by both order volume and average order value.
 
  Most orders are concentrated at lower values, with a long right tail.  
  Winsorization helps reduce the impact of extreme values while preserving all observations.

  Refund rates vary slightly across countries.  
  Bootstrap analysis indicates that the difference is small and not statistically significant.

### Caveats

- The dataset contains a small number of missing values, mainly in time-related fields.
- Some countries have relatively few observations, which may affect the reliability of comparisons.
- The analysis is based on historical data and may not generalize to future behavior.
- Bootstrap confidence intervals describe uncertainty but do not imply causality.
- Results should be interpreted as exploratory rather than definitive conclusions.

In [16]:

df = pd.read_parquet(data1)

print("\n")
print("Row count:", len(df))
print("\n")
print("Data types:")
print(df.dtypes.head(15))
print("\n")
print("Top missing columns:")
print(df.isna().sum().sort_values(ascending=False).head(10))



Row count: 5


Data types:
order_id               string[python]
user_id                         Int64
amount                        float64
quantity                      float64
created_at        datetime64[ns, UTC]
status                 string[python]
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
dtype: object


Top missing columns:
amount           1
amount_winsor    1
quantity         1
created_at       1
hour             1
dow              1
month            1
year             1
date             1
order_id         0
dtype: int64


In [17]:
fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=30,
    title="Distribution of Order Amounts (Winsorized)"
)

save_figure(fig, figures_dir / "amount_distribution.png")
fig.show()

NameError: name 'save_figure' is not defined

In [ ]:
revenue = (
    df.groupby("month", dropna=False)
      .agg(
          revenue=("amount", "sum"),
          orders=("order_id", "count")
      )
      .reset_index()
      .sort_values("month")
)

revenue

,month,revenue,orders
0,2025-12,45.5,4
1,<NA>,100.0,1


In [ ]:
fig = px.bar(
    revenue,
    x="month",
    y="revenue",
    title="Revenue Over Time by month",
    text_auto=".3s"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Total Revenue",
    title=1
)

save_figure(fig, figures_dir / "revenue_over_time.png")
fig

In [ ]:
revenue2 = (
    df.groupby("country", dropna=False)
      .agg(
          n=("order_id", "count"),
          revenue=("amount", "sum")
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)

revenue2

NameError: name 'df' is not defined

In [5]:
from pathlib import Path
import pandas as pd

data_file = Path("../data/processed/analytics_table.parquet")
df = pd.read_parquet(data_file)

df.head()

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,A0001,1,12.5,1.0,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.500
1,A0002,2,8.0,2.0,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.135
2,A0003,3,NaN,1.0,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,NaN
3,A0004,1,25.0,NaN,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.000
4,A0005,4,100.0,1.0,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,97.750


In [18]:
revenue2 = (
    df.groupby("country", dropna=False)
      .agg(
          n=("order_id", "count"),
          revenue=("amount", "sum")
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)

revenue2

,country,n,revenue
1,SA,4,145.5
0,AE,1,0.0


In [20]:
from pathlib import Path

def save_figure(fig, file_path: Path, scale: int = 2) -> None:
    file_path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(file_path), scale=scale)

In [23]:
from pathlib import Path

figures_dir = Path("reports/figures")
figures_dir.mkdir(parents=True, exist_ok=True)

In [24]:
fig = px.bar(
    revenue2,
    x="country",
    y="revenue",
    title="Revenue by Country",
    text_auto=".2s"
)

fig.update_layout(
    xaxis_title="Country",
    yaxis_title="Total Revenue",
    title_x=1
)

save_figure(fig, figures_dir / "revenue_by_country.png")
fig

In [25]:

fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=30,
    title="Histogram of Order Amounts"
)

fig.update_layout(
    xaxis_title="Order Amount (winsorized)",
    yaxis_title="Number of orders",
    title_x=1
)

save_figure(fig, figures_dir / "amount_hist_winsor.png")
fig.show()

In [27]:
df["is_refund"] = df["status_clean"].eq("refund")

In [ ]:
import numpy as np


group1= df[df["country"] == "SA"]["is_refund"]
group2 = df[df["country"] == "AE"]["is_refund"]


print("SA count:", len(group1))
print("AE count:", len(group2)) 

rng = np.random.default_rng(0)
diffs = []

for _ in range(200):
    sa = rng.choice(a, size=len(a), replace=True)
    ae = rng.choice(b, size=len(b), replace=True)
    diffs.append(sa.mean() - ae.mean())


diff_mean = group1.mean() - group2.mean()
ci_low = np.percentile(diffs, 2.5)
ci_high = np.percentile(diffs, 97.5)

print("diff mean =", diff_mean)
print("ci low =", ci_low)
print("ci high:", ci_high)

diff_mean: -1.0
ci_low: -1.0
ci_high: -1.0


In [32]:
import pandas as pd

df = pd.read_parquet("../data/processed/analytics_table.parquet")

df.head(5)
df.shape
df.dtypes
df.isna().sum().sort_values(ascending=False)


amount            1
amount_winsor     1
quantity          1
created_at        1
hour              1
dow               1
month             1
year              1
date              1
order_id          0
quantity__isna    0
status_clean      0
status            0
user_id           0
amount__isna      0
country           0
signup_date       0
dtype: int64

In [33]:
revenue_time = (
    df.groupby("month", dropna=False)
      .agg(
          revenue=("amount", "sum"),
          n=("order_id", "count")
      )
      .reset_index()
)

revenue_time


,month,revenue,n
0,2025-12,45.5,4
1,<NA>,100.0,1


In [34]:
import plotly.express as px

fig = px.line(
    revenue_time,
    x="month",
    y="revenue",
    markers=True,
    title="Revenue Over Time"
)

fig